# Efficient Yelp API Calls - Charlotte, North Carolina's Pizza

* James Belk
* 03/26/2023

## Imports, Installs, and Functions

### Imports

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Functions

In [1]:
# Functions
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

## Credentials

In [3]:
with open('/Users/james/.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

## Define Search

In [4]:
LOCATION = 'Charlotte, NC'
TERM = 'Pizza'

In [5]:
JSON_FILE = 'Data/results_in_progress_Charlotte_pizza.json'
JSON_FILE

'Data/results_in_progress_Charlotte_pizza.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Charlotte_pizza.json not found. Saving empty list to file.


In [8]:
# Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
# set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


## The Search

In [9]:
# user our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                               term = TERM,
                               offset = n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
total_results = results['total']
total_results

1800

In [10]:
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

90

In [13]:
previous_results.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(previous_results,f)

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/90 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## The Dataset

In [15]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,99j5vhhtFJIefXDLsmA29Q,inizio-pizza-napoletana-charlotte,Inizio Pizza Napoletana,https://s3-media2.fl.yelpcdn.com/bphoto/wVJYAR...,False,https://www.yelp.com/biz/inizio-pizza-napoleta...,605,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.0659114, 'longitude': -80.7704...",[delivery],$$,"{'address1': '10620 Providence Rd', 'address2'...",+17048468909,(704) 846-8909,16210.367205
1,9hKhrI5bcz0Mvc6upSdM4w,the-crust-pizza-parktowne-village-charlotte,The Crust Pizza - ParkTowne Village,https://s3-media3.fl.yelpcdn.com/bphoto/mA04DW...,False,https://www.yelp.com/biz/the-crust-pizza-parkt...,43,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.1719194, 'longitude': -80.8477...",[],$$,"{'address1': '1600 E Woodlawn Rd', 'address2':...",+17048005584,(704) 800-5584,3999.877258
2,HrR_2-HUE6cqnoz1zMN9AQ,pizzeria-omaggio-charlotte-4,Pizzeria Omaggio,https://s3-media2.fl.yelpcdn.com/bphoto/9R8Kbb...,False,https://www.yelp.com/biz/pizzeria-omaggio-char...,386,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.2133541062856, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '1055 Metropolitan Ave', 'address...",+17043700777,(704) 370-0777,1097.049335
3,fxd1d4mep5j8XbE_DzecKA,geno-ds-pizza-charlotte,Geno D's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/qnqeRz...,False,https://www.yelp.com/biz/geno-ds-pizza-charlot...,121,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 35.227779647089925, 'longitude': ...",[delivery],$$,"{'address1': '224 E 7th St', 'address2': '', '...",+17049005202,(704) 900-5202,2712.150454
4,v72XMTh-dI3L6kHZSRpzpw,capishe-real-italian-kitchen-charlotte,Capishe Real Italian Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/JUEbzZ...,False,https://www.yelp.com/biz/capishe-real-italian-...,550,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.217279, 'longitude': -80.848077}","[delivery, pickup]",$$,"{'address1': '500 E Morehead St', 'address2': ...",+19808199494,(980) 819-9494,2227.261378


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,ieD9ARRRTs0eLjgl_LcbFw,jon-s-tavern-and-grill-pageland,Jon’s Tavern & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/w4M54F...,False,https://www.yelp.com/biz/jon-s-tavern-and-gril...,2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 34.78501420501474, 'longitude': -...",[],NaN,"{'address1': '1302 Van Lingle Mungo Blvd', 'ad...",+18436726175,(843) 672-6175,61214.797593
996,YAYFcYcJ2dXeuKoCFs_W0A,cicis-pizza-rock-hill-rock-hill,Cicis Pizza - Rock Hill,https://s3-media1.fl.yelpcdn.com/bphoto/tdtKah...,False,https://www.yelp.com/biz/cicis-pizza-rock-hill...,35,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 34.9610585348883, 'longitude': -8...","[pickup, delivery]",$,"{'address1': '1807 Cherry Rd', 'address2': 'St...",+18033243433,(803) 324-3433,31492.685323
997,6hZgMg9rnDCC0RBVRlkZwQ,mr-tokyo-japanese-restaurant-charlotte-2,Mr Tokyo Japanese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/q9t4Yq...,False,https://www.yelp.com/biz/mr-tokyo-japanese-res...,668,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 35.0876649, 'longitude': -80.8608...",[delivery],$$,"{'address1': '8334 Pineville Matthew Rd', 'add...",+17045415566,(704) 541-5566,13261.240333
998,XQqugSurkmFjmByzd0ZsLg,pizza-hut-kannapolis-7,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/zqa5N3...,False,https://www.yelp.com/biz/pizza-hut-kannapolis-...,14,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 35.436778932045335, 'longitude': ...",[delivery],$,"{'address1': '6031 Gateway Center Dr', 'addres...",+17047886031,(704) 788-6031,29169.015850
999,W8b7s805VMj469qw0E88DQ,deli-queen-charlotte,Deli Queen,,False,https://www.yelp.com/biz/deli-queen-charlotte?...,1,"[{'alias': 'restaurants', 'title': 'Restaurant...",5.0,"{'latitude': 35.1474495, 'longitude': -80.7456...",[delivery],NaN,"{'address1': '9010 Monroe Rd', 'address2': 'St...",+17048411877,(704) 841-1877,9898.412409


In [16]:
final_df.duplicated(subset='id').sum()

0

In [17]:
final_df.to_csv('Data/final_results_Charlotte_Pizza.csv.gz', compression='gzip', index = False)

In [19]:
display(final_df.info())
display(final_df.head(), final_df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   object 
 1   alias          1000 non-null   object 
 2   name           1000 non-null   object 
 3   image_url      1000 non-null   object 
 4   is_closed      1000 non-null   bool   
 5   url            1000 non-null   object 
 6   review_count   1000 non-null   int64  
 7   categories     1000 non-null   object 
 8   rating         1000 non-null   float64
 9   coordinates    1000 non-null   object 
 10  transactions   1000 non-null   object 
 11  price          789 non-null    object 
 12  location       1000 non-null   object 
 13  phone          1000 non-null   object 
 14  display_phone  1000 non-null   object 
 15  distance       1000 non-null   float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 118.3+ KB


None

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,99j5vhhtFJIefXDLsmA29Q,inizio-pizza-napoletana-charlotte,Inizio Pizza Napoletana,https://s3-media2.fl.yelpcdn.com/bphoto/wVJYAR...,False,https://www.yelp.com/biz/inizio-pizza-napoleta...,605,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.0659114, 'longitude': -80.7704...",[delivery],$$,"{'address1': '10620 Providence Rd', 'address2'...",+17048468909,(704) 846-8909,16210.367205
1,9hKhrI5bcz0Mvc6upSdM4w,the-crust-pizza-parktowne-village-charlotte,The Crust Pizza - ParkTowne Village,https://s3-media3.fl.yelpcdn.com/bphoto/mA04DW...,False,https://www.yelp.com/biz/the-crust-pizza-parkt...,43,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.1719194, 'longitude': -80.8477...",[],$$,"{'address1': '1600 E Woodlawn Rd', 'address2':...",+17048005584,(704) 800-5584,3999.877258
2,HrR_2-HUE6cqnoz1zMN9AQ,pizzeria-omaggio-charlotte-4,Pizzeria Omaggio,https://s3-media2.fl.yelpcdn.com/bphoto/9R8Kbb...,False,https://www.yelp.com/biz/pizzeria-omaggio-char...,386,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.2133541062856, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '1055 Metropolitan Ave', 'address...",+17043700777,(704) 370-0777,1097.049335
3,fxd1d4mep5j8XbE_DzecKA,geno-ds-pizza-charlotte,Geno D's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/qnqeRz...,False,https://www.yelp.com/biz/geno-ds-pizza-charlot...,121,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 35.227779647089925, 'longitude': ...",[delivery],$$,"{'address1': '224 E 7th St', 'address2': '', '...",+17049005202,(704) 900-5202,2712.150454
4,v72XMTh-dI3L6kHZSRpzpw,capishe-real-italian-kitchen-charlotte,Capishe Real Italian Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/JUEbzZ...,False,https://www.yelp.com/biz/capishe-real-italian-...,550,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.217279, 'longitude': -80.848077}","[delivery, pickup]",$$,"{'address1': '500 E Morehead St', 'address2': ...",+19808199494,(980) 819-9494,2227.261378


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,ieD9ARRRTs0eLjgl_LcbFw,jon-s-tavern-and-grill-pageland,Jon’s Tavern & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/w4M54F...,False,https://www.yelp.com/biz/jon-s-tavern-and-gril...,2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 34.78501420501474, 'longitude': -...",[],NaN,"{'address1': '1302 Van Lingle Mungo Blvd', 'ad...",+18436726175,(843) 672-6175,61214.797593
996,YAYFcYcJ2dXeuKoCFs_W0A,cicis-pizza-rock-hill-rock-hill,Cicis Pizza - Rock Hill,https://s3-media1.fl.yelpcdn.com/bphoto/tdtKah...,False,https://www.yelp.com/biz/cicis-pizza-rock-hill...,35,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 34.9610585348883, 'longitude': -8...","[pickup, delivery]",$,"{'address1': '1807 Cherry Rd', 'address2': 'St...",+18033243433,(803) 324-3433,31492.685323
997,6hZgMg9rnDCC0RBVRlkZwQ,mr-tokyo-japanese-restaurant-charlotte-2,Mr Tokyo Japanese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/q9t4Yq...,False,https://www.yelp.com/biz/mr-tokyo-japanese-res...,668,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 35.0876649, 'longitude': -80.8608...",[delivery],$$,"{'address1': '8334 Pineville Matthew Rd', 'add...",+17045415566,(704) 541-5566,13261.240333
998,XQqugSurkmFjmByzd0ZsLg,pizza-hut-kannapolis-7,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/zqa5N3...,False,https://www.yelp.com/biz/pizza-hut-kannapolis-...,14,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 35.436778932045335, 'longitude': ...",[delivery],$,"{'address1': '6031 Gateway Center Dr', 'addres...",+17047886031,(704) 788-6031,29169.015850
999,W8b7s805VMj469qw0E88DQ,deli-queen-charlotte,Deli Queen,,False,https://www.yelp.com/biz/deli-queen-charlotte?...,1,"[{'alias': 'restaurants', 'title': 'Restaurant...",5.0,"{'latitude': 35.1474495, 'longitude': -80.7456...",[delivery],NaN,"{'address1': '9010 Monroe Rd', 'address2': 'St...",+17048411877,(704) 841-1877,9898.412409
